In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import numpy as np
import datetime
import cv2
import time

import matplotlib.pyplot as plt
import scipy, scipy.io.wavfile

import sys
# the mock-0.3.1 dir contains testcase.py, testutils.py & mock.py
sys.path.append('/home/cat/Downloads/spinnaker_python-1.27.0.48-Ubuntu18.04-cp37-cp37m-linux_x86_64/Examples/Python3/')

from Specgram.Specgram import Specgram



Autosaving every 180 seconds


In [63]:
def get_isi(P, thresh):

    data = P
    print (P.shape)
    data = data.sum(0)-data.sum(0).mean(0)

    scale = P.shape[1]/200.
    t=np.arange(P.shape[1])/200.

    # find peaks
    #thresh = 15000
    idx = np.where(data>thresh)[0]
    idx_corrected=[]
    idx_corrected.append(idx[0])
    for k in range(1, idx.shape[0], 1):
        if idx[k]-idx[k-1]>5:
            idx_corrected.append(idx[k])

    idx = np.array(idx_corrected)

    # find peaks that different various amounts
    diffs = idx[1:]-idx[:-1]
    print (diffs.shape)

    # look for thresholds that are bigger than some minmum width; 
    idx2 = np.where(np.logical_and(diffs!=200,diffs>3,diffs!=199))[0]
    print (diffs[idx2])

    #ax=plt.subplot(211)
    print (idx[idx2+1]/200.)

    return idx[idx2+1]/200.

def get_pings(data, thresh):
    
    idx = np.where(data>thresh)[0]
    #print (idx)
    pings = []
    pings.append(idx[0])
    for k in range(1, idx.shape[0],1):
        if (idx[k]-idx[k-1])<5000:
            pass
        else:
            pings.append(idx[k])

    pings = np.array(pings)/96000.
    print (pings)
    
    return (pings)

In [86]:
from scipy.io import wavfile

#fnames = [
#'/mnt/53abab64-8e58-435f-ae3f-45613b0ecb71/4test/20200513_000104_096.wav',
#'/mnt/53abab64-8e58-435f-ae3f-45613b0ecb71/4test/20200513_000302_096.wav',
#'/mnt/53abab64-8e58-435f-ae3f-45613b0ecb71/4test/20200513_000502_096.wav',
#'/mnt/53abab64-8e58-435f-ae3f-45613b0ecb71/4test/20200513_000702_096.wav']
#fnames = ['/mnt/53abab64-8e58-435f-ae3f-45613b0ecb71/20200513_000104_096.wav',
#'/mnt/53abab64-8e58-435f-ae3f-45613b0ecb71/20200513_000302_096.wav']
fnames = ['/mnt/53abab64-8e58-435f-ae3f-45613b0ecb71/20200513_000104_096.wav']
ctr=0
for fname in fnames:
    fs, data = wavfile.read(fname)
    print (data.shape, fs)

    thresh=10000
    pings = get_pings(data, thresh)
    print (pings)
    diffs = pings[1:]-pings[:-1]
    print (diffs)
    
np.save('/home/cat/pings.npy',pings)
#np.save('/home/cat/pings.npy',pings)


(172982272,) 96000
[   8.63321875    9.63336458   10.6335625  ... 1799.45379167 1800.45396875
 1801.45416667]
[   8.63321875    9.63336458   10.6335625  ... 1799.45379167 1800.45396875
 1801.45416667]
[1.00014583 1.00019792 1.00014583 ... 1.00017708 1.00017708 1.00019792]


In [85]:
idx = np.where(diffs<.9)[0]
print (idx)

[ 131  132  147  148  207  208 1757]


In [84]:
fig=plt.figure()
ax=plt.subplot(111)
y = np.histogram(diffs, np.arange(0.,2,0.01))
plt.plot(y[1][1:],y[0])
plt.title("Inter-beep difference (seconds)",fontsize=20)
#ax.set_yscale('log')
#ax.set_ylim(0.1,2000)
plt.show()

In [61]:
fnames = ['/mnt/53abab64-8e58-435f-ae3f-45613b0ecb71/20200513_000104_096.wav',
'/mnt/53abab64-8e58-435f-ae3f-45613b0ecb71/20200513_000302_096.wav']

fs, data = wavfile.read(fnames[1])
print (data.shape, fs)
plt.plot(data[:500000])

plt.show()

(5931008,) 96000


[ 0.6009375   1.60111458  2.6013125   3.60146875  4.60166667  5.6018125
  6.60196875  7.60215625  8.602375    9.60255208 10.60271875 11.60292708]
[1.00017708 1.00019792 1.00015625 1.00019792 1.00014583 1.00015625
 1.0001875  1.00021875 1.00017708 1.00016667 1.00020833]


In [12]:
P, extent = Specgram(data = data, sampfreq = fs, 
                         p0=-300, f0=0.1, f1=fs, 
                         width=0.01, tres = 0.005)

Computing regular fft specgram


In [21]:

idx = get_isi(P, thresh=3500)
print (idx)

(5,)
[2942 2510 2491 1102 2085]
[15.935 28.485 40.94  46.45  56.875]
[15.935 28.485 40.94  46.45  56.875]


In [14]:
print (P.shape, extent)
plt.imshow(P,extent=extent, aspect='auto')
plt.xlabel("Time (sec)", fontsize=20)
plt.show()

(480, 12321) (0.0, 61.61065625, 100.0, 48000.0)


In [ ]:
# LOAD DATA SETS
fnames = np.loadtxt('/mnt/53abab64-8e58-435f-ae3f-45613b0ecb71/may_7/audio/names.txt',dtype=np.str)
times = []
ctr = 0
for fname in fnames:
    audio_rec = np.load(fname)
    P, extent = Specgram(data = audio_rec.squeeze(), sampfreq = 384000, 
                         p0=-300, f0=0.1, f1=384000, 
                         width=0.01, tres = 0.005)


    time = get_isi(P)
    times.append(time)
    ctr+=1
    print ("")
    print ("")

In [82]:
times3600 = [[120.465, 180.475, 240.46, 300.455, 420.435, 480.445, 540.455, 600.465, 660.48, 780.46, 840.47, 900.46, 960.425, 1020.395, 1140.345, 1200.295, 1260.29, 1320.185, 1380.115, 1500.065, 1559.95, 1619.93, 1679.845, 1739.775, 1799.73, 1919.605, 1979.585, 2039.465, 2099.395, 2159.355, 2279.24, 2339.135, 2399.02, 2458.92, 2518.935, 2638.795, 2698.605, 2758.6, 2818.515, 2878.365, 2998.29, 3058.205, 3118.145, 3178.09, 3237.975, 3357.855, 3417.705, 3477.68, 3537.58], [120.405, 180.415, 240.41, 300.42, 420.415, 480.39, 540.36, 600.37, 660.3399999999999, 780.345, 840.355, 900.365, 960.35, 1020.365, 1140.385, 1200.395, 1260.405, 1320.395, 1380.405, 1500.365, 1560.35, 1620.3600000000001, 1680.35, 1740.345, 1860.365, 1920.375, 1980.3600000000001, 2040.34, 2100.35, 2220.37, 2280.38, 2340.33, 2400.345, 2460.355, 2580.33, 2640.32, 2700.33, 2760.345, 2820.335, 2940.32, 3000.305, 3060.315, 3120.235, 3180.25, 3300.07, 3359.995, 3419.91, 3479.825, 3539.71], [120.47, 180.415, 240.39, 300.355, 420.335, 480.345, 540.335, 600.345, 660.36, 780.36, 840.375, 900.385, 960.395, 1020.38, 1140.325, 1200.34, 1260.35, 1320.3600000000001, 1380.37, 1500.365, 1560.38, 1620.3899999999999, 1680.4, 1740.385, 1860.39, 1920.4, 1980.41, 2040.4, 2100.365, 2220.37, 2280.38, 2340.345, 2400.315, 2460.27, 2580.255, 2640.245, 2700.22, 2760.195, 2820.21, 2940.23, 3000.24, 3060.225, 3120.205, 3180.17, 3300.01, 3359.79, 3419.755, 3479.675, 3539.565], [120.53, 180.54, 240.53, 300.505, 420.435, 480.43, 540.44, 600.45, 660.46, 780.445, 840.435, 900.425, 960.415, 1020.425, 1140.425, 1200.435, 1260.445, 1320.455, 1380.465, 1500.485, 1560.475, 1620.4850000000001, 1680.495, 1740.505, 1860.505, 1920.465, 1980.48, 2040.49, 2100.5, 2220.48, 2280.47, 2340.48, 2400.475, 2460.485, 2580.475, 2640.47, 2700.48, 2760.49, 2820.5, 2940.52, 3000.515, 3060.525, 3120.455, 3180.4049999999997, 3300.19, 3360.1, 3419.965, 3479.915, 3539.88], [120.5, 180.51, 240.52, 300.51, 420.515, 480.48, 540.49, 600.5, 660.51, 780.5, 840.51, 900.52, 960.53, 1020.54, 1140.56, 1200.55, 1260.535, 1320.52, 1380.4850000000001, 1500.475, 1560.485, 1620.495, 1680.505, 1740.5149999999999, 1860.515, 1920.48, 1980.465, 2040.475, 2100.45, 2220.445, 2280.455, 2340.45, 2400.46, 2460.43, 2580.45, 2640.46, 2700.45, 2760.445, 2820.455, 2940.455, 3000.445, 3060.43, 3120.395, 3180.34, 3300.305, 3360.3, 3420.27, 3480.195, 3540.12], [120.485, 180.47, 240.435, 300.4, 420.385, 480.355, 540.3, 600.31, 660.3199999999999, 780.3, 840.31, 900.325, 960.31, 1020.3199999999999, 1140.305, 1200.295, 1260.31, 1320.32, 1380.33, 1500.35, 1560.36, 1620.37, 1680.38, 1740.375, 1860.35, 1920.36, 1980.37, 2040.385, 2100.375, 2220.365, 2280.375, 2340.315, 2400.325, 2460.335, 2580.29, 2640.285, 2700.295, 2760.305, 2820.295, 2940.26, 3000.27, 3060.28, 3120.2, 3180.19, 3300.01, 3359.94, 3419.835, 3479.8, 3539.715], [120.545, 180.53, 240.52, 300.465, 420.4, 480.37, 540.38, 600.355, 660.345, 780.365, 840.375, 900.385, 960.395, 1020.405, 1140.41, 1200.4, 1260.41, 1320.42, 1380.43, 1500.41, 1560.42, 1620.43, 1680.44, 1740.43, 1860.415, 1920.425, 1980.435, 2040.445, 2100.46, 2220.48, 2280.49, 2340.5, 2400.47, 2460.48, 2580.5, 2640.47, 2700.48, 2760.495, 2820.505, 2940.525, 3000.515, 3060.53, 3120.45, 3180.445, 3300.225, 3360.22, 3420.035, 3479.98, 3539.96], [120.535, 180.525, 240.49, 300.48, 420.43, 480.415, 540.395, 600.385, 660.395, 780.395, 840.405, 900.37, 960.38, 1020.39, 1140.395, 1200.35, 1260.335, 1320.345, 1380.355, 1500.33, 1560.34, 1620.33, 1680.295, 1740.305, 1860.27, 1920.28, 1980.27, 2040.26, 2100.27, 2220.245, 2280.255, 2340.265, 2400.275, 2460.285, 2580.285, 2640.295, 2700.265, 2760.275, 2820.285, 2940.285, 3000.265, 3060.055, 3120.065, 3179.995, 3239.955, 3359.835, 3419.665, 3479.675, 3539.59], [120.57, 179.75, 239.71, 299.675, 359.685, 479.6, 539.565, 599.5550000000001, 659.565, 719.5550000000001, 839.58, 899.59, 959.6, 1019.61, 1079.62, 1199.64, 1259.65, 1319.66, 1379.67, 1439.685, 1559.675, 1619.685, 1679.695, 1739.705, 1799.7, 1919.695, 1979.71, 2039.7, 2099.71, 2159.69, 2279.71, 2339.72, 2399.705, 2459.695, 2519.705, 2639.685, 2699.68, 2759.69, 2819.7, 2879.71, 2999.53, 3059.34, 3119.315, 3179.285, 3239.23, 3359.09, 3419.005, 3478.96, 3538.925], [120.565, 180.56, 240.57, 300.46, 420.39, 480.4, 540.365, 600.375, 660.365, 780.385, 840.38, 900.365, 960.36, 1020.37, 1140.345, 1200.32, 1260.33, 1320.34, 1380.305, 1500.305, 1560.29, 1620.285, 1680.295, 1740.305, 1860.31, 1920.305, 1980.315, 2040.325, 2100.335, 2220.34, 2280.35, 2340.36, 2400.37, 2460.365, 2580.385, 2640.395, 2700.405, 2760.415, 2820.425, 2940.425, 3000.42, 3060.41, 3120.405, 3180.33, 3300.175, 3360.05, 3419.935, 3479.945, 3539.89], [120.53, 180.54, 240.53, 300.54, 420.455, 480.425, 540.38, 600.365, 660.375, 780.355, 840.365, 900.34, 960.33, 1020.3399999999999, 1140.345, 1200.335, 1260.345, 1320.355, 1380.365, 1500.345, 1560.355, 1620.35, 1680.3600000000001, 1740.37, 1860.33, 1920.34, 1980.325, 2040.31, 2100.325, 2220.285, 2280.295, 2340.285, 2400.295, 2460.305, 2580.275, 2640.285, 2700.295, 2760.285, 2820.295, 2940.315, 3000.305, 3060.25, 3120.1, 3180.035, 3239.895, 3359.65, 3419.595, 3479.545, 3539.5], [120.47, 180.46, 240.415, 300.4, 420.335, 480.28, 540.275, 600.285, 660.275, 780.275, 840.265, 900.275, 960.235, 1020.245, 1140.265, 1200.275, 1260.285, 1320.295, 1380.305, 1500.285, 1560.295, 1620.31, 1680.32, 1740.33, 1860.29, 1920.3, 1980.315, 2040.305, 2100.295, 2220.295, 2280.31, 2340.32, 2400.33, 2460.34, 2580.29, 2640.285, 2700.295, 2760.235, 2820.23, 2940.25, 3000.26, 3060.27, 3120.205, 3180.17, 3300.025, 3359.99, 3419.965, 3479.945, 3539.89], [120.58, 180.57, 240.58, 300.57, 420.59000000000003, 480.58, 540.59, 600.605, 660.615, 780.61, 840.62, 900.63, 960.64, 1020.655, 1140.65, 1200.64, 1260.65, 1320.635, 1380.645, 1500.645, 1560.585, 1620.595, 1680.605, 1740.595, 1860.615, 1920.585, 1980.595, 2040.59, 2100.575, 2220.58, 2280.59, 2340.6, 2400.61, 2460.625, 2580.61, 2640.62, 2700.615, 2760.625, 2820.62, 2940.64, 3000.63, 3060.64, 3120.6, 3180.555, 3300.39, 3360.305, 3420.22, 3480.195, 3540.12], [120.465, 180.46, 240.45, 300.39, 420.35, 480.345, 540.33, 600.31, 660.3199999999999, 780.305, 840.315, 900.31, 960.3199999999999, 1020.3299999999999, 1140.335, 1200.345, 1260.355, 1320.35, 1380.3600000000001, 1500.36, 1560.37, 1620.335, 1680.345, 1740.355, 1860.165, 1920.135, 1980.13, 2040.085, 2100.0950000000003, 2220.055, 2280.04, 2340.015, 2400.005, 2460.015, 2580.02, 2640.03, 2700.04, 2760.05, 2820.06, 2940.065, 3000.05, 3060.01, 3119.805, 3179.8, 3239.715, 3359.61, 3419.48, 3479.435, 3539.305], [120.58, 180.55, 240.56, 300.555, 420.575, 480.585, 540.595, 600.605, 660.615, 780.6, 840.61, 900.62, 960.63, 1020.595, 1140.58, 1200.57, 1260.52, 1320.5149999999999, 1380.525, 1500.545, 1560.555, 1620.57, 1680.58, 1740.59, 1860.595, 1920.565, 1980.555, 2040.565, 2100.58, 2220.535, 2280.525, 2340.535, 2400.545, 2460.555, 2580.535, 2640.53, 2700.515, 2760.525, 2820.535, 2940.51, 3000.52, 3060.425, 3120.4, 3180.275, 3300.075, 3360.03, 3419.905, 3479.825, 3539.82], [120.585, 180.595, 240.605, 300.59, 420.53499999999997, 480.52, 540.515, 600.5, 660.51, 780.475, 840.465, 900.455, 960.465, 1020.46, 1140.48, 1200.49, 1260.505, 1320.5149999999999, 1380.525, 1500.52, 1560.53, 1620.52, 1680.53, 1740.54, 1860.56, 1920.555, 1980.565, 2040.575, 2100.57, 2220.59, 2280.6, 2340.61, 2400.62, 2460.61, 2580.605, 2640.62, 2700.61, 2760.605, 2820.575, 2940.58, 3000.59, 3060.555, 3120.52, 3180.38, 3300.17, 3360.16, 3420.04, 3480.0, 3539.94], [120.495, 180.485, 240.485, 300.475, 420.48, 480.49, 540.505, 600.475, 660.485, 780.505, 840.515, 900.51, 960.52, 1020.53, 1140.535, 1200.515, 1260.5, 1320.51, 1380.505, 1500.45, 1560.46, 1620.47, 1680.48, 1740.495, 1860.455, 1920.465, 1980.445, 2040.455, 2100.465, 2220.28, 2280.255, 2340.135, 2400.045, 2459.985, 2519.815, 2639.65, 2699.595, 2759.55, 2819.465, 2879.455, 2999.325, 3059.25, 3119.245, 3179.24, 3239.14, 3359.03, 3418.96, 3478.925, 3538.855], [120.51, 180.5, 240.515, 300.505, 420.43, 480.4, 540.38, 600.365, 660.36, 780.355, 840.37, 900.38, 960.39, 1020.38, 1140.405, 1200.415, 1260.4, 1320.3899999999999, 1380.38, 1500.38, 1560.37, 1620.3600000000001, 1680.35, 1740.3600000000001, 1860.365, 1920.375, 1980.345, 2040.31, 2100.32, 2220.315, 2280.325, 2340.32, 2400.33, 2460.34, 2580.345, 2640.355, 2700.365, 2760.375, 2820.385, 2940.39, 3000.4, 3060.39, 3120.265, 3180.26, 3300.05, 3360.01, 3419.97, 3479.91, 3539.85]]

#np.save('/home/cat/times3600.npy', times3600)

time1 = np.load('/home/cat/Downloads/times_3600.npy')
time2 = np.load('/home/cat/Downloads/may_8_times_3600.npy')

time = np.vstack((time1, time2))
print (time.shape)

(79, 49)


In [83]:
timesfixed = np.array(time.copy())
for k in range(len(timesfixed)):
    timesfixed[k] = timesfixed[k]-timesfixed[k][0]+120
    print (k, timesfixed[k])


0 [ 120.     180.01   239.995  299.99   419.97   479.98   539.99   600.
  660.015  779.995  840.005  899.995  959.96  1019.93  1139.88  1199.83
 1259.825 1319.72  1379.65  1499.6   1559.485 1619.465 1679.38  1739.31
 1799.265 1919.14  1979.12  2039.    2098.93  2158.89  2278.775 2338.67
 2398.555 2458.455 2518.47  2638.33  2698.14  2758.135 2818.05  2877.9
 2997.825 3057.74  3117.68  3177.625 3237.51  3357.39  3417.24  3477.215
 3537.115]
1 [ 120.     180.01   240.005  300.015  420.01   479.985  539.955  599.965
  659.935  779.94   839.95   899.96   959.945 1019.96  1139.98  1199.99
 1260.    1319.99  1380.    1499.96  1559.945 1619.955 1679.945 1739.94
 1859.96  1919.97  1979.955 2039.935 2099.945 2219.965 2279.975 2339.925
 2399.94  2459.95  2579.925 2639.915 2699.925 2759.94  2819.93  2939.915
 2999.9   3059.91  3119.83  3179.845 3299.665 3359.59  3419.505 3479.42
 3539.305]
2 [ 120.     179.945  239.92   299.885  419.865  479.875  539.865  599.875
  659.89   779.89   839.905  899.9

In [84]:
times1D = np.hstack(timesfixed)

y = np.histogram(times1D, np.arange(0,3600,0.010))
for k in range(59):
    ax=plt.subplot(6,10,k+2)
    #plt.bar(y[1][:-1], y[0],0.009)
    plt.plot(y[1][:-1], y[0])
    plt.xlim((k+2)*60-5,(k+2)*60+5)
    #plt.ylabel("5sec beep",fontsize=20)
    #plt.title("")
    ax.tick_params(axis = 'both', which = 'major', labelsize = 8, pad=0.9)
plt.xlabel("Time (sec)")
plt.show()

In [101]:
fig=plt.figure()
y = np.histogram(times1D, np.arange(0,3600,0.010))
plt.plot(y[1][:-1], y[0])
plt.xlim(3535,3542)
plt.plot([3540,3540],[0,3],'r--',c='red')
plt.ylim(top=5)
plt.xlabel("seconds")
plt.title("Distribution of drift from "+str(time.shape[0])+" recordings across 3600 seconds\n"+\
           "red line location of beep",fontsize=20)
#ax.tick_params(axis = 'both', which = 'major', labelsize = 8, pad=0.9)
# print (y[0].shape)
# fig =plt.figure()
# snippit = y[0][353000:]
# print (snippit.shape, snippit.sum(0))
# plt.plot(snippit)
plt.show()

In [79]:
timesfixed[0]
diffs = timesfixed[0][1:]-timesfixed[0][:-1]-60
print (diffs)

y = np.histogram(diffs, np.arange(-1,1,.0250))
plt.plot(y[1][:-1], y[0])
#plt.xlim(59,63542)
#plt.plot([3540,3540],[0,3],'r--',c='red')
#plt.ylim(top=3)
plt.xlabel("seconds")
plt.title("Distribution of drift from 18 recordings across 3600 seconds\n"+\
           "red line location of beep",fontsize=20)
#ax.tick_params(axis = 'both', which = 'major', labelsize = 8, pad=0.9)
plt.show()

[ 1.0000e-02 -1.5000e-02 -5.0000e-03  5.9980e+01  1.0000e-02  1.0000e-02
  1.0000e-02  1.5000e-02  5.9980e+01  1.0000e-02 -1.0000e-02 -3.5000e-02
 -3.0000e-02  5.9950e+01 -5.0000e-02 -5.0000e-03 -1.0500e-01 -7.0000e-02
  5.9950e+01 -1.1500e-01 -2.0000e-02 -8.5000e-02 -7.0000e-02 -4.5000e-02
  5.9875e+01 -2.0000e-02 -1.2000e-01 -7.0000e-02 -4.0000e-02  5.9885e+01
 -1.0500e-01 -1.1500e-01 -1.0000e-01  1.5000e-02  5.9860e+01 -1.9000e-01
 -5.0000e-03 -8.5000e-02 -1.5000e-01  5.9925e+01 -8.5000e-02 -6.0000e-02
 -5.5000e-02 -1.1500e-01  5.9880e+01 -1.5000e-01 -2.5000e-02 -1.0000e-01]


In [33]:
times_copy = times.copy()
# for k in range(len(times_copy)):
#     print (k, times_copy[k])
times_copy[7] = times_copy[7][1:]
#44 [ 5.55  30.555 53.275 55.505]
#45 [ 5.54  30.545 38.035 55.53 ]
times_copy[44] = [ 5.55, 30.555,  55.505]
times_copy[45] = [5.54,  30.545, 55.53 ]
times_copy = np.vstack(times_copy)
#print (times_copy)

times_copy = (times_copy.T - times_copy[:,0]).T+5.0
bin_width = 0.010
y = np.histogram(times_copy, np.arange(0,60,0.010))
ax=plt.subplot(3,1,1)
plt.bar(y[1][:-1], y[0],bin_width*.9)
#plt.plot(y[1][:-1], y[0])

plt.xlim(4.9,5.1)
plt.ylabel("5sec beep",fontsize=20)

ax=plt.subplot(3,1,2)
#plt.bar(y[1][:-1], y[0],0.009)
#plt.plot(y[1][:-1], y[0])
plt.bar(y[1][:-1], y[0],bin_width*.9)
plt.xlim(29.9,30.1)
plt.ylabel("30sec beep",fontsize=20)

ax=plt.subplot(3,1,3)
#plt.plot(y[1][:-1], y[0])
plt.bar(y[1][:-1], y[0],bin_width*.9)
plt.xlim(54.9,55.1)
plt.ylabel("55sec beep",fontsize=20)
plt.xlabel("Time (sec)", fontsize=20)
plt.show()

In [16]:
# visualize audio
print ("rec: ", audio_rec.shape)

P, extent = Specgram(data = audio_rec.squeeze(), sampfreq = 384000, 
                     p0=-300, f0=0.1, f1=384000, 
                     width=0.01, tres = 0.005)

print (P.shape, extent)
plt.imshow(P,extent=extent, aspect='auto')
plt.xlabel("Time (sec)", fontsize=20)
plt.show()


rec:  (23040000, 1)
Computing regular fft specgram
(1920, 11999) (0.0, 59.99999739583333, 100.0, 192000.0)


In [17]:
data = P #[:1920,:120000]
rec_len = 60

print (data.sum(0).mean())
data = data.sum(0)-data.sum(0).mean(0)

scale = P.shape[1]/200.
t=np.arange(P.shape[1])/200.

# find peaks
thresh = 18000
idx = np.where(data>thresh)[0]
idx_corrected=[]
idx_corrected.append(idx[0])
for k in range(1, idx.shape[0], 1):
    if idx[k]-idx[k-1]>5:
        idx_corrected.append(idx[k])
    
idx = np.array(idx_corrected)
#print (idx)

# find peaks that different various amounts
diffs = idx[1:]-idx[:-1]
print (diffs.shape)

# look for thresholds that are bigger than some minmum width; 
idx2 = np.where(np.logical_and(diffs!=200,diffs>3,diffs!=199))[0]
print (diffs[idx2])

ax=plt.subplot(211)
plt.plot(t, data)
#plt.plot([0,119999/200.],[thresh,thresh],c='red')
for id_ in idx_corrected:
    plt.plot([id_/200.,id_/200.],[0,30000],c='green')
for id_ in idx[idx2+1]:
    plt.plot([id_/200.,id_/200.],[0,30000],c='red')

# manually define a new patch 
#patch = mpatches.Patch(color='grey', label='1sec beep')
#labels.append(patch) 
#plt.legend(handles=labels, loc='upper center')
print (idx[idx2+1]/200.)

#plt.title()

ax=plt.subplot(212)
y = np.histogram(diffs, bins = np.arange(10,400,.001))
plt.plot(y[1][:-1]/scale,y[0])
ax.set_yscale('log')
#plt.title("600 second audio recording of 1sec beeps", fontsize=20)
plt.xlabel("Inter-beep interval distributions (seconds)",fontsize=20)
plt.ylabel("# of frames apart by x-axis amount",fontsize=20)
plt.show()

-328144.05501826375
(3,)
[ 307 5797 5001]
[ 1.535 30.52  55.525]


In [ ]:
-328144.05501826375
(3,)
[ 307 5797 5001]
[ 1.535 30.52  55.525]

-328619.41143776797
(3,)
[ 291 5767 4993]
[ 1.455 30.29  55.255]
